In [12]:
import json

In [13]:
vocab_raw = json.load(open('../dict/vocab_list.json', 'r'))

#clean _num_ 
num_keys = [i for i in vocab_raw.keys() if (i.find('_num_') != -1)]
for i in num_keys:
    vocab_raw.pop(i)

#new key list
vocab_keys = [u'_num_']
vocab_keys.extend(vocab_raw.keys())

#reserve index 0: padding, index 1: _num_
vocab_dict = {k:v for k, v in zip(vocab_keys, range(2, len(vocab_keys)+2))}

In [14]:
vocab_dict

{u'limited': 3,
 u'hitch': 4,
 u'four': 5,
 u'asian': 6,
 u'captain': 7,
 u'integrity': 8,
 u'btec': 9,
 u'consider': 1318,
 u'whose': 11,
 u'calculate': 12,
 u'aug': 13,
 u'electricity': 14,
 u'tweet': 15,
 u'disability': 81,
 u'under': 17,
 u'sorry': 18,
 u'worth': 19,
 u'collaborate': 20,
 u'risk': 21,
 u'dollar': 1322,
 u'regional': 23,
 u'every': 24,
 u'affect': 25,
 u'encounter': 26,
 u'amran': 27,
 u'school': 28,
 u'progression': 29,
 u'good': 1890,
 u'cmc': 30,
 u'wednesday': 31,
 u'convenience': 32,
 u'enhance': 33,
 u'ups': 34,
 u'confidence': 2042,
 u'enjoy': 35,
 u'chew': 36,
 u'force': 37,
 u'japanese': 38,
 u'nair': 39,
 u'second': 40,
 u'chee': 41,
 u'chen': 42,
 u'even': 43,
 u'neo': 44,
 u'nel': 45,
 u'worsen': 46,
 u'profitability': 47,
 u'asia': 48,
 u'spokesman': 49,
 u'haze': 50,
 u'above': 51,
 u'conduct': 52,
 u'supplier': 53,
 u'new': 54,
 u'net': 55,
 u'officially': 56,
 u'specialist': 57,
 u'reporter': 58,
 u'never': 59,
 u'here': 60,
 u'protection': 61,
 u'ch

In [15]:
def word2Int(wd):
    try:
        return(vocab_dict[wd])
    except KeyError:
        return 1

print(word2Int(u'nuk'), word2Int(u'train'))

(1, 2063)


In [16]:
import pandas as pd

#Cheng's labeled data
train_df = pd.read_csv('../NER-label/training-set.csv', quotechar='\'', names=['POS', 'TEXT', 'ENTITY'])

#labeled data
#train_df = pd.read_csv('/home/NER-label/NER-marked/marked-training-data.csv', quotechar='\"', index_col=False, names=['POS', 'TEXT', 'ENTITY'])

train_df.shape

(10488, 3)

In [17]:
from collections import Counter

pos_keys = Counter(train_df['POS']).keys()
pos2Int_ = {k:v for k, v in zip(pos_keys, range(len(pos_keys)))}
pos2Int = lambda x:pos2Int_[x]

In [18]:
#tagging cleaning
train_df.ix[train_df['ENTITY'] == 'LOC', 'ENTITY'] = u'O'
train_df.ix[train_df['ENTITY'] == ' O', 'ENTITY'] = u'O'
train_df.ix[train_df['ENTITY'] == ' ORG', 'ENTITY'] = u'ORG'
train_df.ix[train_df['ENTITY'] == '0RG', 'ENTITY'] = u'ORG'
train_df.ix[train_df['ENTITY'] == 'DATE/ORG', 'ENTITY'] = u'DATE'
train_df.ix[train_df['ENTITY'] == 'FAC', 'ENTITY'] = u'STATION'
train_df.ix[train_df['ENTITY'] == 'GPE', 'ENTITY'] = u'COUNTRY'
train_df.ix[train_df['ENTITY'] == 'MONEY/ORG', 'ENTITY'] = u'MONEY'
train_df.ix[train_df['ENTITY'] == 'OT', 'ENTITY'] = u'LINE'
train_df.ix[train_df['ENTITY'] == 'ORDINAL', 'ENTITY'] = u'O'
train_df.ix[train_df['ENTITY'] == 'ROAD/STATION', 'ENTITY'] = u'STATION'
train_df.ix[train_df['ENTITY'] == 'CARDINAL', 'ENTITY'] = u'GNUM'
train_df.ix[train_df['ENTITY'] == 'CARDINAL', 'ENTITY'] = u'O'

In [19]:
entity_keys = Counter(train_df['ENTITY']).keys()
entity2Int_ = {k:v for k, v in zip(entity_keys, range(len(entity_keys)))}
entity2Int = lambda x:entity2Int_[x]

In [20]:
print(entity2Int_)

{nan: 0, 'INTERCHANGE': 1, 'SERVICE': 2, 'TITLE': 3, 'DATE': 4, 'STATION': 5, 'DURATION': 6, 'LINE': 7, 'AREA': 8, 'BUS': 9, 'LOCATION': 10, 'ROAD': 11, u'GNUM': 12, 'PERCENT': 13, 'O': 14, 'DEPOT': 15, 'PRODUCT': 16, 'NORP': 17, 'MONEY': 18, 'PERSON': 19, 'TIME': 20, 'ORG': 21, 'EVENT': 22}


In [21]:
print(pos2Int_)

{nan: 0, 'ADV': 1, 'NOUN': 2, 'ADP': 3, 'PUNCT': 10, 'PROPN': 5, 'DET': 6, 'SYM': 7, 'INTJ': 8, 'PRON': 4, 'PART': 9, 'X': 12, 'NUM': 11, 'CONJ': 13, 'ADJ': 14, 'VERB': 15}


In [22]:
#save entity key-value pair
json.dump(entity2Int_, open('./Model/entityDict.json', 'w'))
json.dump(pos2Int_, open('./Model/posDict.json', 'w'))

In [23]:
train_df['TEXT'] = train_df['TEXT'].astype('string')

sent_break = train_df[train_df['TEXT'] == 'nan'].index

In [24]:
#replace the number with '_num_' and words
train_df.ix[train_df['POS'] == 'NUM', 'TEXT'] = u'_num_'
train_df.ix[train_df['TEXT'] == '\'ve', 'TEXT'] = u'have'
train_df.ix[train_df['TEXT'] == '\'m', 'TEXT'] = u'am'
train_df.ix[train_df['TEXT'] == '\'re', 'TEXT'] = u'are'
train_df.ix[train_df['TEXT'] == '\'s', 'TEXT'] = u'is'

In [25]:
tmp_v = [-1]
tmp_v.extend(sent_break.values)
tmp_v.append(train_df.shape[0])

POS_DATA = []
TEXT_DATA = []
ENTITY_DATA = []

for i in range(len(tmp_v)-1):
    start_i = tmp_v[i] + 1
    end_i = tmp_v[i+1]
    
    if((end_i-start_i)>3): #remove empty string
        pos_v = train_df['POS'][start_i:end_i]
        if('NUM' in pos_v.values) or ('PROPN' in pos_v.values):
            POS_DATA.append(pos_v)
            TEXT_DATA.append(train_df['TEXT'][start_i:end_i])
            ENTITY_DATA.append(train_df['ENTITY'][start_i:end_i])

print(len(ENTITY_DATA))            

322


In [26]:
from textacy import Doc

POS_MAPPING_DATA = [[pos2Int(i) for i in il] for il in POS_DATA]
ENTITY_MAPPING_DATA = [[entity2Int(i) for i in il] for il in ENTITY_DATA]

def text2lemma(pds):
    assert pds.shape[0] > 1
    try:
        doc_v = Doc(u' '.join(pds))
        for i in doc_v:
            yield(i.lemma_)
    except(RuntimeError):
        print(pds)

TEXT_MAPPING_DATA = [[word2Int(i) for i in text2lemma(il)] for il in TEXT_DATA]
print(ENTITY_DATA[0:2])
print(TEXT_DATA[0:2])

[0            O
1            O
2            O
3          ORG
4            O
5            O
6          BUS
7            O
8          BUS
9            O
10           O
11           O
12           O
13        DATE
14           O
15        DATE
16        DATE
17           O
18           O
19           O
20    LOCATION
21    LOCATION
22           O
23           O
24           O
25           O
26        ROAD
27        ROAD
28        ROAD
29        ROAD
30           O
31           O
32           O
33           O
34           O
35           O
36           O
37           O
38           O
39           O
Name: ENTITY, dtype: object, 41       O
42       O
43       O
44       O
45       O
46       O
47       O
48       O
49       O
50       O
51    ROAD
52    ROAD
53       O
54       O
Name: ENTITY, dtype: object]
[0           The
1        routes
2            of
3          SMRT
4           bus
5      services
6         _num_
7           and
8         _num_
9          will
10           be
11      am

In [94]:
print("Number fo sentences: "+str(len(TEXT_MAPPING_DATA)))
print((ENTITY_DATA))

Number fo sentences: 322
<type 'list'>


In [40]:
import keras
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Activation, Dense, Dropout, merge, Input
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.layers.recurrent import LSTM

Using TensorFlow backend.


In [41]:
#multiple input model (word embedding + pos tagging)
import numpy as np

vocab_size = 2+len(vocab_dict.keys())
maxlen = 3 # must be odd
embedding_dim = 200
hidden_dim = 500
output_dim = len(entity2Int_.keys())
pos_dim = len(pos2Int_.keys())

#model input (word embedding)
word_input = Input(shape=(maxlen, ), name='word_input')
word_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, init='uniform')(word_input)

#model input(pos tagging)
pos_input = Input(shape=(maxlen, ), name='pos_input')
pos_embedding = Embedding(input_dim=pos_dim, output_dim=pos_dim, init='uniform')(pos_input)
combined_input = merge([word_embedding, pos_embedding], mode='concat', concat_axis=2)

lstm1 = LSTM(512, return_sequences= , dropout_W=0.2, dropout_U=0.2, input_shape=(maxlen, embedding_dim+pos_dim))(combined_input)
lstm2 = LSTM(512, return_sequences=False)(lstm1)
drop1 = Dropout(0.5)(lstm2)
output = Dense(output_dim, activation='softmax', name='output')(drop1)
model = Model(input=[word_input, pos_input], output=output)


#combined model
#model = Sequential()
#model.add(embedding)
#model.add(LSTM(512, return_sequences=True, dropout_W=0.2, dropout_U=0.2, input_shape=(maxlen, embedding_dim)))
#model.add(LSTM(512, return_sequences=False))
#model.add(Dropout(0.5))
#model.add(Dense(output_dim))
#model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(embeddings_initializer="uniform", output_dim=200, input_dim=2613)`
/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(embeddings_initializer="uniform", output_dim=16, input_dim=16)`
/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/fay/anaconda/lib/python2.7/site-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: Update

In [42]:
#sequential RNN model

##
#vocab_size = 2+len(vocab_dict.keys())
#maxlen = 9 # must be odd
#embedding_dim = 200
#hidden_dim = 500
#output_dim = len(entity2Int_.keys())

#model 1
#embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen, init='uniform')

#combined model
#model = Sequential()
#model.add(embedding)
#model.add(LSTM(512, return_sequences=True, dropout_W=0.2, dropout_U=0.2, input_shape=(maxlen, embedding_dim)))
#model.add(LSTM(512, return_sequences=False))
#model.add(Dropout(0.5))
#model.add(Dense(output_dim))
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [43]:
def contextwin(l, win):
    '''
    win :: int corresponding to the size of the window
    given a list of indexes composing a sentence

    l :: array containing the word indexes

    it will return a list of list of indexes corresponding
    to context windows surrounding each word in the sentence
    '''
    assert (win % 2) == 1
    assert win >= 1
    l = list(l)

    lpadded = win // 2 * [-1] + l + win // 2 * [-1]
    out = [lpadded[i:(i + win)] for i in range(len(l))]

    #print(len(l))
    assert len(out) == len(l)
    return out

#contextwin(TEXT_MAPPING_DATA[1], 9)

In [44]:
#prepare training input
TRAIN_X = []
SIZE_X = []
for i in TEXT_MAPPING_DATA:
    tmpv = contextwin(i, maxlen)
    SIZE_X.append(len(tmpv))    
    TRAIN_X.extend(tmpv)

print(np.array(TRAIN_X).shape)


(9420, 3)


In [45]:
#prepare training output
def int2array(int_v, length):
    result_v = [0]*length
    if(int_v != -1):
        result_v[int_v] = 1
    return result_v

TRAIN_Y = []
SIZE_Y = []
TRAIN_Y_1 = []
print(ENTITY_MAPPING_DATA[0])
for i in ENTITY_MAPPING_DATA:
    tmpv = map(lambda x:int2array(x, output_dim), i)
    SIZE_Y.append(len(tmpv))
    TRAIN_Y.extend(tmpv)
    TRAIN_Y_1.append(tmpv)

print(ENTITY_MAPPING_DATA[0])
print(TRAIN_Y[0],TRAIN_Y[1],TRAIN_Y[2],TRAIN_Y[3]) 


[14, 14, 14, 21, 14, 14, 9, 14, 9, 14, 14, 14, 14, 4, 14, 4, 4, 14, 14, 14, 10, 10, 14, 14, 14, 14, 11, 11, 11, 11, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
[14, 14, 14, 21, 14, 14, 9, 14, 9, 14, 14, 14, 14, 4, 14, 4, 4, 14, 14, 14, 10, 10, 14, 14, 14, 14, 11, 11, 11, 11, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])


In [46]:
TRAIN_A = []
SIZE_A = []
TRAIN_A_1 = []

for i in POS_MAPPING_DATA:
    tmpv = contextwin(i, maxlen)    
    SIZE_A.append(len(tmpv))
    TRAIN_A.extend(tmpv)
    TRAIN_A_1.append(tmpv)
    
print(POS_MAPPING_DATA[0])
print(TRAIN_A[0],TRAIN_A[1])    





[6, 2, 3, 5, 2, 2, 11, 13, 11, 15, 15, 15, 3, 5, 10, 5, 11, 10, 10, 3, 5, 5, 3, 6, 2, 3, 5, 5, 9, 5, 15, 15, 15, 9, 3, 2, 13, 14, 2, 10]
([-1, 6, 2], [6, 2, 3])


In [47]:
print("POS",np.array(TRAIN_A).shape)
print("TEXT",np.array(TRAIN_X).shape)
print("ENTITY",np.array(TRAIN_Y).shape)

print("POS ORI", np.array(POS_MAPPING_DATA).shape)
print("TEXT ORI", np.array(TEXT_MAPPING_DATA).shape)
print("ENTITY ORI", np.array(ENTITY_MAPPING_DATA).shape)

('POS', (8813, 3))
('TEXT', (9420, 3))
('ENTITY', (8813, 23))
('POS ORI', (322,))
('TEXT ORI', (322,))
('ENTITY ORI', (322,))


In [48]:
#prepare training input
TRAIN_X = []
SIZE_X = []
index_X = 0
TRAIN_X_1 = []
for i in TEXT_MAPPING_DATA:
    tmpv = contextwin(i, maxlen)
    TRAIN_X.extend(tmpv)
    SIZE_X.append(len(tmpv))
    TRAIN_X_1.append(tmpv)

print(np.array(TRAIN_X).shape)
print(np.array(SIZE_X).shape)
print(np.array(TRAIN_X_1).shape)

(9420, 3)
(322,)
(322,)


In [49]:
i_not_in = []
for i in range(len(SIZE_X)):
    if SIZE_X[i] != SIZE_Y[i]:
        ##print(i, SIZE_X[i], SIZE_Y[i])
        i_not_in.append(i)
i_in = [i for i in range(len(SIZE_X)) if i not in i_not_in]
print(i_not_in)
print(i_in)

TRAIN_X_2 = []
TRAIN_Y_2 = []
TRAIN_A_2 = []
for i in range(len(SIZE_X)):
    if i in i_in:
        TRAIN_X_2.extend(TRAIN_X_1[i])
        TRAIN_Y_2.extend(TRAIN_Y_1[i])
        TRAIN_A_2.extend(TRAIN_A_1[i])

print(np.array(TRAIN_X).shape)
print(np.array(TRAIN_Y).shape)
print(np.array(TRAIN_A).shape)

print(np.array(TRAIN_X_2).shape)
print(np.array(TRAIN_Y_2).shape)
print(np.array(TRAIN_A_2).shape)

[0, 3, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 23, 24, 28, 31, 33, 35, 37, 41, 42, 44, 45, 46, 48, 50, 53, 57, 58, 67, 69, 70, 73, 74, 75, 76, 79, 82, 83, 86, 87, 89, 90, 91, 98, 100, 101, 102, 103, 104, 107, 112, 113, 120, 121, 122, 124, 130, 131, 132, 133, 134, 135, 138, 139, 140, 143, 145, 146, 152, 153, 154, 155, 158, 162, 165, 168, 169, 170, 171, 174, 177, 178, 183, 184, 185, 187, 191, 192, 193, 194, 199, 200, 201, 202, 203, 204, 205, 206, 208, 209, 210, 212, 219, 220, 222, 229, 230, 231, 232, 233, 235, 236, 238, 241, 242, 244, 246, 249, 250, 251, 252, 254, 255, 256, 257, 258, 259, 260, 262, 264, 265, 267, 268, 272, 273, 275, 276, 277, 281, 282, 285, 288, 290, 293, 297, 299, 300, 301, 302, 305, 312, 313, 320, 321]
[1, 2, 4, 5, 13, 20, 21, 22, 25, 26, 27, 29, 30, 32, 34, 36, 38, 39, 40, 43, 47, 49, 51, 52, 54, 55, 56, 59, 60, 61, 62, 63, 64, 65, 66, 68, 71, 72, 77, 78, 80, 81, 84, 85, 88, 92, 93, 94, 95, 96, 97, 99, 105, 106, 108, 109, 110, 111, 114, 115, 116, 117, 118, 119

In [50]:
print("POS",np.array(TRAIN_A)[0])
print("POS",(TRAIN_A)[0])
print("TEXT",np.array(TRAIN_X_2)[0])
print("ENTITY",np.array(TRAIN_Y_2)[0])

('POS', array([-1,  6,  2]))
('POS', [-1, 6, 2])
('TEXT', array([  -1,  490, 2377]))
('ENTITY', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]))


In [51]:
TRAIN_X_2_NP =  np.array(TRAIN_X_2) 
TRAIN_X_2_NP[TRAIN_X_2_NP == -1] =0
print("TEXT",np.array(TRAIN_X_2_NP)[0])

TRAIN_A_2_NP =  np.array(TRAIN_A_2) 
TRAIN_A_2_NP[TRAIN_A_2_NP == -1] =0
print("TEXT",np.array(TRAIN_A_2_NP)[0])

('TEXT', array([   0,  490, 2377]))
('TEXT', array([0, 6, 2]))


In [52]:
#print(len(TRAIN_x), len(TRAIN_Y))
#pred_v = model.predict(TRAIN_X)
#pred_v.shape
#print(model.predict_classes(TRAIN_X))
#print(ENTITY_MAPPING_DATA[1])

In [53]:

model.fit({'word_input':TRAIN_X_2_NP, 'pos_input':TRAIN_A_2_NP}, {'output':np.array(TRAIN_Y_2)}, nb_epoch=20, validation_split=0.1, shuffle=True)

/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 3757 samples, validate on 418 samples
Epoch 1/20
3757/3757 [==============================] - 32s - loss: 0.7534 - acc: 0.8411 - val_loss: 0.5116 - val_acc: 0.8445
Epoch 2/20
3757/3757 [==============================] - 28s - loss: 0.4481 - acc: 0.8805 - val_loss: 0.4178 - val_acc: 0.8947
Epoch 3/20
3757/3757 [==============================] - 28s - loss: 0.3364 - acc: 0.9079 - val_loss: 0.4404 - val_acc: 0.8995
Epoch 4/20
3757/3757 [==============================] - 28s - loss: 0.2629 - acc: 0.9271 - val_loss: 0.3725 - val_acc: 0.9067
Epoch 5/20
3757/3757 [==============================] - 28s - loss: 0.2067 - acc: 0.9388 - val_loss: 0.4357 - val_acc: 0.9091
Epoch 6/20
3757/3757 [==============================] - 28s - loss: 0.1677 - acc: 0.9532 - val_loss: 0.4861 - val_acc: 0.9091
Epoch 7/20
3757/3757 [==============================] - 28s - loss: 0.1316 - acc: 0.9614 - val_loss: 0.4980 - val_acc: 0.8971
Epoch 8/20
3757/3757 [==============================] - 27s - loss: 0.1

In [122]:

model.fit({'word_input':np.array(TRAIN_X)}, {'output':np.array(TRAIN_Y)}, nb_epoch=5, validation_split=0.1, shuffle=True)

/Users/fay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


ValueError: Input arrays should have the same number of samples as target arrays. Found 9420 input samples and 8813 target samples.

In [54]:
def saveModel(model):
    model_json = model.to_json()
    with open("./Model/model-keras1.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("./Model/model-keras1.h5")
    print("Saved model to disk")

saveModel(model)

Saved model to disk
